# CoreScale results for "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Configure router for CoreScale scenario

> CoreScale: The “at scale" setting with a bottleneck bandwidth
of 10 Gbps, 1000 to 5000 competing flows, and a 375MB
buffer

We implement CoreScale at the router using a `tc htb` rate limiting queue. Because of [limitations of the software implementation](https://unix.stackexchange.com/questions/704918/has-10-gbps-through-linux-tc-qdiscs-ever-been-solved), we use hardware offload on the Mellanox interface to achieve the full 10 Gbps.

First, make sure this feature is turned on:

In [ ]:
router_node.execute('sudo ethtool  -K ' + router_egress_name + ' hw-tc-offload on')
router_node.execute('sudo ethtool  -k ' + router_egress_name + " | grep hw-tc-offload" )

Now, we should be able to configure the 10 Gbps queue.

First delete any existing queue (don't worry if there is an error, it means there was not!)

In [ ]:
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

Then set one up, with hardware offload:

In [ ]:
router_node.execute("sudo tc qdisc replace dev " + router_egress_name + " root handle 1: htb default 3 offload")
router_node.execute("sudo tc class add dev " + router_egress_name + " parent 1: classid 1:3 htb rate 10Gbit")
router_node.execute("sudo tc qdisc add dev " + router_egress_name + " parent 1:3 handle 3: bfifo limit 375MB")

## Check network capacity

This time, we should get close to **10 Gbps** through the router.

Now start the `iperf3` flows:

In [ ]:
for n in receiver_nodes:
    n.execute("sudo killall iperf3; iperf3 -s -1 -f g -D --logfile validate.dat")

In [ ]:
import time
for i, n in enumerate(sender_nodes):
    n.execute_thread("sudo killall iperf3; iperf3 -t 60 -c 10.10.2.1"+str(i))
time.sleep(185)

In [ ]:
tputs = []
for n in receiver_nodes:
    (nout, nerr) = n.execute("tail --lines=2 validate.dat | grep receiver | awk '{print $7}'")
    tputs.append(float(nout.strip()))
print("Sum throughput is: %f Gbps" % sum(tputs))

A single pair of endpoints should be able to get about 3 to 7 Gbps.

In [ ]:
receiver_nodes[0].execute("iperf3 -s -1 -f g -D")
sender_nodes[0].execute("iperf3 -t 60 -i 60 -c 10.10.2.10")

## Check network delay

We should still get sub-ms latency across the router.

In [ ]:
for i, n in enumerate(sender_nodes):
    n.execute("ping -c 5 10.10.2.1"+str(i)+" | grep rtt")